### Reference.
Torch-Demos : [digit classifier](https://github.com/torch/demos/blob/master/train-a-digit-classifier/train-on-mnist.lua)

In [1]:
require 'torch'
require 'nn'
require 'optim'
require 'image'
require 'dataset-mnist'

In [2]:
-- Setting Hyperparameters
batchSize = 50
learningRate = 0.05
momentum = 0
numIter = 20

-- Fix Seed
torch.manualSeed(1)

-- Setting worker
torch.setnumthreads(2)
print('<torch> set nb of threads to ' .. torch.getnumthreads())

-- Use Floats
torch.setdefaulttensortype('torch.FloatTensor')

<torch> set nb of threads to 1	


In [3]:
-- Define model to Train
-- On the 10-class Classification Problem
-- Geometry : width and height of input images

classes = {'1', '2' ,'3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9', '10'}
geometry = {32, 32}

In [4]:
numTrain = 60000
numTest = 10000

-- Create training set and normalize
trainData = mnist.loadTrainSet(numTrain, geometry)
trainData:normalizeGlobal(mean, std)

-- Create test set and normalize
testData = mnist.loadTestSet(numTest, geometry)
testData:normalizeGlobal(mean, std)

<mnist> done	


<mnist> done	


In [5]:
print(trainData)
print(testData)

{
  data : FloatTensor - size: 60000x1x32x32
  normalize : function: 0x0ce96930
  labels : ByteTensor - size: 60000
  normalizeGlobal : function: 0x0ce968f8
  size : function: 0x0ce969d8
}
{
  data : FloatTensor - size: 10000x1x32x32
  normalize : function: 0x0ce977c0
  labels : ByteTensor - size: 10000
  normalizeGlobal : function: 0x0ce97788
  size : function: 0x0ce97868
}


In [6]:
-- 2 Layer Neural Network
model = nn.Sequential()

model:add(nn.Reshape(32*32))
model:add(nn.Linear(32*32, 2048))
model:add(nn.Tanh())
model:add(nn.Linear(2048, #classes))
model:add(nn.LogSoftMax())

criterion = nn.ClassNLLCriterion()

In [7]:
-- Retrieve parameters and gradients
parameters, gradParameters = model:getParameters()

confusion = optim.ConfusionMatrix(classes)
trainHistory = {}
testHistory = {}

In [8]:
-- Train function
function train(dataset)
    -- epoch tracker
    epoch = epoch or 1

    -- local vars
    local time = sys.clock()

    -- do one epoch
    print('<trainer> on training set:')
    print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
    for t = 1, numTrain, batchSize do
        -- create mini batch
        local inputs = torch.Tensor(batchSize,1,geometry[1],geometry[2])
        local targets = torch.Tensor(batchSize)
        local k = 1
        for i = t, math.min(t+batchSize-1, numTrain) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        -- create closure to evaluate f(X) and df/dX
        local feval = function(x)
            -- just in case:
            collectgarbage()

            -- get new parameters
            if x ~= parameters then
                parameters:copy(x)
            end

            -- reset gradients
            gradParameters:zero()

            -- evaluate function for complete mini batch
            local outputs = model:forward(inputs)
            local f = criterion:forward(outputs, targets)

            -- estimate df/dW
            local df_do = criterion:backward(outputs, targets)
            model:backward(inputs, df_do)

            -- update confusion
            for i = 1,batchSize do
                confusion:add(outputs[i], targets[i])
            end

            -- return f and df/dX
            return f,gradParameters
        end

        -- Perform SGD step:
        sgdState = sgdState or {
            learningRate = learningRate,
            momentum = momentum,
            learningRateDecay = 5e-7
        }
        optim.sgd(feval, parameters, sgdState)

        -- disp progress
        xlua.progress(t, numTrain)
    end
   
    -- time taken
    time = sys.clock() - time
    time = time / numTrain
    print("<trainer> time to learn 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
    print(confusion)
    table.insert(trainHistory, confusion.totalValid * 100)
    confusion:zero()

    -- next epoch
    epoch = epoch + 1
end

In [9]:
-- test function
function test(dataset)
    -- local vars
    local time = sys.clock()

    -- test over given dataset
    print('<trainer> on testing Set:')
    for t = 1, dataset:size(), batchSize do
        -- disp progress
        xlua.progress(t, dataset:size())

        -- create mini batch
        local inputs = torch.Tensor(batchSize, 1, geometry[1], geometry[2])
        local targets = torch.Tensor(batchSize)
        local k = 1
        for i = t,math.min(t+batchSize-1, dataset:size()) do
            -- load new sample
            local sample = dataset[i]
            local input = sample[1]:clone()
            local _,target = sample[2]:clone():max(1)
            target = target:squeeze()
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end

        -- test samples
        local preds = model:forward(inputs)

        -- confusion:
        for i = 1, batchSize do
            confusion:add(preds[i], targets[i])
        end
    end

    -- timing
    time = sys.clock() - time
    time = time / dataset:size()
    print("<trainer> time to test 1 sample = " .. (time*1000) .. 'ms')

    -- print confusion matrix
    print(confusion)
    table.insert(testHistory, confusion.totalValid * 100)
    confusion:zero()
end

In [10]:
for i = 1, numIter do
    train(trainData)
    test(testData)
end

<trainer> on training set:	
<trainer> online epoch # 1 [batchSize = 50]	


Progress: 23901 / 60000	


Progress: 50451 / 60000	


<trainer> time to learn 1 sample = 0.21535853147507ms	
ConfusionMatrix:
[[    5679       1      38      25      13      60      44       4      49      10]   95.880% 	[class: 1]
 [       1    6508      39      32       7      33       7      15      85      15]   96.529% 	[class: 2]
 [      49      71    5291      98      90      35      90      87     121      26]   88.805% 	[class: 3]
 [      32      39     127    5408       6     219      39      72     124      65]   88.207% 	[class: 4]
 [      13      30      45       8    5375       7      63      18      33     250]   92.006% 	[class: 5]
 [      71      42      33     216      72    4650     108      29     137      63]   85.778% 	[class: 6]
 [      51      27      64       5      65      78    5584       3      39       2]   94.356% 	[class: 7]
 [      30      56      84      24      68      11       8    5780      13     191]   92.259% 	[class: 8]
 [      33     118      88     166      33     170      49      22    5073      

<trainer> time to test 1 sample = 0.086409401893616ms	
ConfusionMatrix:
[[     971       0       1       1       0       3       2       1       1       0]   99.082% 	[class: 1]
 [       0    1121       2       2       1       1       3       2       3       0]   98.767% 	[class: 2]
 [      11      25     901      40      11       3      10       6      18       7]   87.306% 	[class: 3]
 [       5       3      12     925       1      34       1      10       5      14]   91.584% 	[class: 4]
 [       2       2       3       1     928       0       8       0       4      34]   94.501% 	[class: 5]
 [      10       3       3      13       9     826       5       4      13       6]   92.601% 	[class: 6]
 [      17       3       5       2       9      27     892       1       2       0]   93.111% 	[class: 7]
 [       3      17      20       9      12       2       0     902       3      60]   87.743% 	[class: 8]
 [      10      20       3      36      12      31       6       4     836      

...

Progress: 56151 / 60000	


<trainer> time to learn 1 sample = 0.2189066807429ms	


ConfusionMatrix:
[[    5923       0       0       0       0       0       0       0       0       0]   100.000% 	[class: 1]
 [       0    6737       0       0       2       0       0       3       0       0]   99.926% 	[class: 2]
 [       0       1    5957       0       0       0       0       0       0       0]   99.983% 	[class: 3]
 [       0       0       0    6130       0       0       0       0       0       1]   99.984% 	[class: 4]
 [       0       0       0       0    5840       0       0       1       0       1]   99.966% 	[class: 5]
 [       0       0       0       0       0    5421       0       0       0       0]   100.000% 	[class: 6]
 [       0       0       0       0       0       0    5918       0       0       0]   100.000% 	[class: 7]
 [       0       4       0       0       0       0       0    6261       0       0]   99.936% 	[class: 8]
 [       0       1       0       0       0       0       0       0    5850       0]   99.983% 	[class: 9]
 [       0       0       0

<trainer> time to test 1 sample = 0.081539392471313ms	


ConfusionMatrix:
[[     973       0       1       1       1       0       1       1       2       0]   99.286% 	[class: 1]
 [       0    1127       2       1       0       1       2       1       1       0]   99.295% 	[class: 2]
 [       4       1    1012       1       2       0       2       5       5       0]   98.062% 	[class: 3]
 [       0       0       4     992       0       3       0       3       1       7]   98.218% 	[class: 4]
 [       1       0       2       1     967       0       3       0       0       8]   98.473% 	[class: 5]
 [       2       0       0       8       2     874       2       0       2       2]   97.982% 	[class: 6]
 [       6       3       0       1       4       7     937       0       0       0]   97.808% 	[class: 7]
 [       2       5       6       1       8       0       0     992       5       9]   96.498% 	[class: 8]
 [       3       2       4       4       5       4       1       2     946       3]   97.125% 	[class: 9]
 [       3       2       0   

In [11]:
function table2Tensor(table)
    tensor = torch.Tensor(#table)
    for i = 1, tensor:size()[1] do
        tensor[i] = table[i]
    end
    return tensor
end

In [12]:
Plot = require 'itorch.Plot'

trainValid = table2Tensor(trainHistory)
testValid = table2Tensor(testHistory)

epochCount = torch.Tensor(numIter)
i = 0

epochCount:apply(function()
    i = i + 1
    return i
end)


-- line plots
plot = Plot():line(epochCount, trainValid, 'green', 'Train')
plot:line(epochCount, testValid, 'blue', 'Test')
plot:legend(true)
plot:title('Train/Test TotalValid'):draw()